# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [42]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [43]:
print(os.getcwd())
filepath = os.getcwd() + '/event_data'
for root, dirs, files in os.walk(filepath):
    
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [44]:
full_data_rows_list = [] 
    
for f in file_path_list:

    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
       
        csvreader = csv.reader(csvfile) 
        next(csvreader)
               
        for line in csvreader:
            full_data_rows_list.append(line) 
            
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [45]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [46]:
from cassandra.cluster import Cluster

try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [47]:
try:
    session.execute(
    """
    CREATE KEYSPACE IF NOT EXISTS sparkifly
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
    )
except Exception as e:
    print(e)

#### Set Keyspace

In [48]:
try:
    session.set_keyspace('sparkifly')
except Exception as e:
    print(e)

### Create song_id_session table to match the query below:

#### SELECT artist, song title and song's length from song_id_session where sessionId = 338, and itemInSession  = 4

In [49]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query = "CREATE TABLE IF NOT EXISTS song_id_session "
query = query + "(sessionId int, itemInSession int, artist text, length float,\
                song text, PRIMARY KEY(sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)               

#### Insert data into song_id_session from event_datafile_new.csv file

In [30]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO song_id_session (sessionId, itemInSession, artist, length, song)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId = line
        session.execute(query, (int(sessionId), int(itemInSession), artist, float(length), song))

#### Query to validate data input into the table
#### SELECT artist, song, length FROM song_id_session WHERE sessionId = 338 AND itemInSession = 4

In [31]:
query = "SELECT artist, song, length FROM song_id_session WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


#### creating the song_library_user modeled around the query:
### SELECT firstName, lastName FROM song_library_user WHERE song = 'All Hands Against His Own'

In [41]:
query = "CREATE TABLE IF NOT EXISTS name_song_session_id "
query = query + "(userId int, sessionId int, itemInSession int, artist text, firstName text, \
              lastName text, song text, PRIMARY KEY((userId, sessionId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                

('Unable to complete the operation against any hosts', {<Host: 127.0.0.1 datacenter1>: ConnectionException('Pool is shutdown',)})


#### Inserting data into the name_song_session_id table from the event_datafile_new.csv file

In [33]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO name_song_session_id (userId, sessionId, itemInSession, artist, firstName, lastName, song)"
        query = query + "VALUES(%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(userId), int(sessionId), int(itemInSession), artist, firstName, lastName, song))

Query #2
Return: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [34]:
query = """SELECT artist, song, firstName, lastName FROM name_song_session_id
        WHERE userId = 10 AND sessionId = 182
        order by itemInSession DESC"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.firstName, row.lastName)

creating the song_library_user modeled around the query:
SELECT firstName, lastName FROM song_library_user WHERE song = 'All Hands Against His Own'

In [36]:
query = "CREATE TABLE IF NOT EXISTS song_library_user "
query = query + "(song text, userId int, firstName text, lastName text, PRIMARY KEY(song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)
 
                    

Inserting data from the csv file into the song_library_user table

In [37]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO song_library_user (song, userId, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, ( song, int(userId), firstName, lastName))

Query #3
Return the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [38]:
query = """SELECT firstName, lastName FROM song_library_user
           WHERE song = 'All Hands Against His Own' """
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print(row.first_name, row.last_name)

### Dropping the 3-tables created 

In [ ]:
query = "drop table song_id_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table name_song_session_id"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table song_library_user"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [40]:
session.shutdown()
cluster.shutdown()